In [1]:
import mne
import os
from utils import check_paths
import numpy as np
import pandas as pd
import pickle
from mne.stats import permutation_t_test

import matplotlib.pyplot as plt
%matplotlib qt

Time-frequency

In [36]:
eeg_data_dir = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set'
group = 'O'
subs_dir = os.path.join(eeg_data_dir, group)

tasks = ['_BL', '_MAIN'] # ['_BL', '_MAIN']
task_stages = ['_plan', '_go'] # '_plan' for PLANNING or '_go' for EXECUTION

# Run time-frequency analysis for individual subjects
# task_stage = '_go' # '_plan' or '_go'
baseline_tf = (-0.5, -0.001) # baseline for TF is prior to the event of interest (zscore)

for sub_name in os.listdir(subs_dir): # os.listdir(subs_dir) OR ['s1_pac_sub00']

    print(f'Creating TF maps for {task} task in {sub_name}...')

    preproc_dir = os.path.join(subs_dir, sub_name, 'preproc')
    analysis_dir = os.path.join(preproc_dir, 'analysis')
    # tfr_dir = os.path.join(figs_dir, 'TFR')
    # check_paths(tfr_dir)
    for task in tasks:
        for task_stage in task_stages:
            if task == '_BL':
                block_names = ['']
            else:
                block_names = ['_baseline', '_adaptation']
                
            for block_name in block_names:
                epochs_data_path = os.path.join(analysis_dir, f'{sub_name}{task}_epochs{task_stage}{block_name}-epo.fif')
                epochs = mne.read_epochs(epochs_data_path, preload=True)
                epochs.apply_baseline(baseline=baseline_tf)

                freqs = np.logspace(*np.log10([4, 80]), num=40)
                n_cycles = freqs / 2.0  # different number of cycle per frequency
                power, itc = epochs.compute_tfr(
                    method="morlet",
                    freqs=freqs,
                    n_cycles=n_cycles,
                    average=True,
                    return_itc=True,
                    decim=3,
                )
                power.apply_baseline(baseline=baseline_tf, mode="zscore")

                # Save time-freq analysis output
                power.save(os.path.join(analysis_dir, f"{sub_name}{task}{task_stage}{block_name}_power-tfr.h5"), overwrite=True)
                itc.save(os.path.join(analysis_dir, f"{sub_name}{task}{task_stage}{block_name}_itc-tfr.h5"), overwrite=True)

                # # plot ITC and Power
                # itc_fig = itc.plot(combine='mean', title=f'Target-reaching task: {task_stage}{block_name}. ITC', cmap='magma')[0]
                # power_all_fig = power.plot(mode="zscore", title='Averaged Power Across Channels', combine='mean', cmap='magma')[0]
                # power_motor_fig = power.plot(picks=['C1', 'C3', 'C5'], mode="zscore", title='Averaged Power Across C1-5 Channels', combine='mean', cmap='magma')[0]

                # # # save ITC and Power plots
                # itc_fig.savefig(os.path.join(tfr_dir, f"{sub_name}{task}{task_stage}{block_name}_itc.png"), dpi=300)
                # power_all_fig.savefig(os.path.join(tfr_dir, f"{sub_name}{task}{task_stage}{block_name}_power_all.png"), dpi=300)
                # power_motor_fig.savefig(os.path.join(tfr_dir, f"{sub_name}{task}{task_stage}{block_name}_power_motor.png"), dpi=300)

                # plt.close('all')

Creating TF maps for _BL task in s1_pac_sub12...
Reading D:\BonoKat\research project\# study 1\eeg_data\set\O\s1_pac_sub12\preproc\analysis\s1_pac_sub12_BL_epochs_plan-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 60) active
    Found the data of interest:
        t =    -500.00 ...     500.00 ms
        0 CTF compensation matrices available
Adding metadata with 10 columns
102 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Applying baseline correction (mode: mean)
Applying baseline correction (mode: zscore)
Overwriting existing file.
Overwriting existing file.
Reading D:\BonoKat\research project\# study 1\eeg_data\set\O\s1_pac_sub12\preproc\analysis\s1_pac_sub12_BL_epochs_go-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 60) active
    Found the data of interest:
        t =    -500.00 ...     500.00 ms
        0 CTF compen

Concatenate epochs

In [ ]:
# Concatenate epochs across subjects
# DONE

eeg_data_dir = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set'
group = 'O'
task = '_BL' # ['_BL', '_MAIN']
task_stage = '_go' # '_plan' or '_go'
block_name = '' # ['_baseline', '_adaptation', '']
subs_dir = os.path.join(eeg_data_dir, group)

all_epochs = []

for sub_name in os.listdir(subs_dir): # os.listdir(subs_dir) OR ['s1_pac_sub00']

    preproc_dir = os.path.join(subs_dir, sub_name, 'preproc')
    analysis_dir = os.path.join(preproc_dir, 'analysis')

    epochs_data_path = os.path.join(analysis_dir, f'{sub_name}{task}_epochs{task_stage}{block_name}-epo.fif')
    epochs = mne.read_epochs(epochs_data_path, preload=True)
    all_epochs.append(epochs)

epochs_all_subs = mne.concatenate_epochs(all_epochs)

group_save_path = f'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\{group} group'
epochs_all_subs.save((os.path.join(group_save_path, f"{group}_ALL_subs{task}_epochs{task_stage}{block_name}_ALL-epo.fif")), overwrite=True)

TFR GROUP

In [ ]:
group = 'O'
group_save_path = f'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\{group} group'
figs_dir = os.path.join(group_save_path, 'sensors', 'figs', 'TF')
check_paths(figs_dir)
sub_name = 'ALL_subs'
tasks = ['_BL', '_MAIN'] # ['_BL', '_MAIN']
task_stages = ['_plan', '_go'] # '_plan' for PLANNING or '_go' for EXECUTION

freqs = np.logspace(*np.log10([4, 80]), num=40)
n_cycles = freqs / 2.0  # different number of cycle per frequency
baseline = (-0.5, -0.001)

for task in tasks:
    for task_stage in task_stages:
        if task == '_BL':
            block_names = ['']
        else:
            block_names = ['_baseline', '_adaptation']
        for block_name in block_names:
            epochs_all_subs = mne.read_epochs(os.path.join(group_save_path, f"{group}_{sub_name}{task}_epochs{task_stage}{block_name}_ALL-epo.fif"), preload=True)
            epochs_all_subs.apply_baseline(baseline=baseline, mode="zscore")

            power, itc = epochs_all_subs.compute_tfr(
                method="morlet",
                freqs=freqs,
                n_cycles=n_cycles,
                average=True,
                return_itc=True,
                decim=3,
                n_jobs=-1
            )

            # Save time-freq analysis output
            itc.save(os.path.join(group_save_path, f"{group}_{sub_name}{task}{task_stage}{block_name}_itc-tfr.h5"), overwrite=True)
            power.save(os.path.join(group_save_path, f"{group}_{sub_name}{task}{task_stage}{block_name}_power-tfr.h5"), overwrite=True)

            # plot ITC and Power
            itc_fig = itc.plot(combine='mean',
                                title=f'ITC: {group}{task}{task_stage}{block_name}',
                                cmap='autumn')[0]
            power_all_fig = power.plot(mode="zscore",
                                        title=f'Averaged Power: {group}{task}{task_stage}{block_name}',
                                        combine='mean', cmap='summer')[0]

            # save ITC and Power plots
            itc_fig.savefig(os.path.join(figs_dir,
                                            f"{group}_{sub_name}{task}{task_stage}{block_name}_itc.png"),
                                            dpi=300)
            power_all_fig.savefig(os.path.join(figs_dir,
                                                f"{group}_{sub_name}{task}{task_stage}{block_name}_power_all.png"),
                                                dpi=300)

            # plt.close('all')


TFR group figures

In [37]:
eeg_data_dir = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set'
groups = ['O']
tasks = ['_BL', '_MAIN'] # ['_BL', '_MAIN']
task_stages = ['_plan', '_go']
sub_name = 'ALL_subs'

In [ ]:
for group in groups:

    group_save_path = f'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\{group} group'

    subs_dir = os.path.join(eeg_data_dir, group)
    subs = os.listdir(subs_dir)
    sensor_figs_path = os.path.join(eeg_data_dir, f'{group} group', 'sensors')
    tfr_figs_save_path = os.path.join(sensor_figs_path, 'figs', 'TF')
    check_paths(tfr_figs_save_path)

    for task in tasks:
        if task == '_BL':
            block_names = ['']
        else:
            block_names = ['_baseline', '_adaptation']

        for task_stage in task_stages:
            for block_name in block_names:
                print(f'Processing {group} group, {task}{task_stage}{block_name}...')


                epochs_all_subs = mne.read_epochs(os.path.join(
                    group_save_path, f"{group}_{sub_name}{task}_epochs{task_stage}{block_name}_ALL-epo.fif"
                    ), preload=True)
                #! Change baseline for prestim period (if not applied yet)
                epochs_all_subs.apply_baseline(baseline=(-0.5, -0.001))
 
                freqs = np.logspace(*np.log10([4.0, epochs_all_subs.info['lowpass']]), num=40)
                n_cycles = freqs / 2.0  # different number of cycle per frequency
                baseline_tf = (epochs_all_subs.times.min(), -0.001)

                ### COMPUTE POWER TFR AND ITC ###
                power, itc = epochs_all_subs.compute_tfr(
                    method="morlet",
                    freqs=freqs,
                    n_cycles=n_cycles,
                    average=True,
                    return_itc=True,
                    decim=3,
                    n_jobs=-1
                )
                power.apply_baseline(baseline=baseline_tf, mode="zscore")

                # plot ITC and Power
                itc_fig = itc.plot(combine='mean',
                                    title=f'ITC: {group}{task}{task_stage}{block_name}',
                                    cmap='autumn')[0]
                power_all_fig = power.plot(mode="zscore",
                                            title=f'Averaged Power: {group}{task}{task_stage}{block_name}',
                                            combine='mean', cmap='summer')[0]

                ax1, ax2 = itc_fig.axes[0], power_all_fig.axes[0]   # main TFR axis

                quadmesh_itc = ax1.collections[0]  # the heatmap
                quadmesh_itc.set_clim(vmin=0., vmax=0.1)

                quadmesh_power = ax2.collections[0]  # the heatmap
                quadmesh_power.set_clim(vmin=-4, vmax=4)

                # save ITC and Power plots
                itc_fig.savefig(os.path.join(tfr_figs_save_path,
                                             f"{group}_{sub_name}{task}{task_stage}{block_name}_itc.png"),
                                             dpi=300)
                power_all_fig.savefig(os.path.join(tfr_figs_save_path,
                                                   f"{group}_{sub_name}{task}{task_stage}{block_name}_power_all.png"),
                                                   dpi=300)

# plt.close('all')


TF group stats

In [38]:
eeg_data_dir = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set'
analysis_dir = os.path.join(eeg_data_dir, f'{group} group', 'sensors', 'analysis')
tfr_group_dir = os.path.join(analysis_dir, 'tfr')
tf_stats_figs_save_path = f'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\{group} group\\sensors\\figs\\TF_stats'
check_paths(tfr_group_dir, tf_stats_figs_save_path)

group = 'O'
tasks = ['_BL', '_MAIN'] # ['_BL', '_MAIN']
task_stages = ['_plan', '_go'] # '_plan' for PLANNING or '_go' for EXECUTION

subs_dir = os.path.join(eeg_data_dir, group)

for task in tasks:
    for task_stage in task_stages:
        if task == '_BL':
            block_names = ['']
        else:
            block_names = ['_baseline', '_adaptation']
        for block_name in block_names:
            power_data = []
            itc_data = []

            for sub_name in os.listdir(subs_dir):

                preproc_dir = os.path.join(subs_dir, sub_name, 'preproc')
                analysis_dir = os.path.join(preproc_dir, 'analysis')    

                # Read TFR and ITC data
                power = mne.time_frequency.read_tfrs(fname=os.path.join(analysis_dir, f"{sub_name}{task}{task_stage}{block_name}_power-tfr.h5"))
                itc = mne.time_frequency.read_tfrs(fname=os.path.join(analysis_dir, f"{sub_name}{task}{task_stage}{block_name}_itc-tfr.h5"))

                power_data.append(power.get_data()) # power is a list with one element, so we take the first element
                # itc_zscore = zscore(itc.get_data(), axis=0, nan_policy='omit') # zscore test - p-values too high
                itc_data.append(itc.get_data()) # itc is a list with one element

            ### POWER TFR  STATS ###
            power_data = np.array(power_data) # shape (n_subjects, n_channels, n_freqs, n_times)
            # Shape before averaging = (n_subjects, n_channels, n_freqs, n_times)
            power_data_ave = np.mean(power_data, axis=(0,1)) # average across subs and channels
            # Average across channels but keep subjects
            power_data_subs = np.mean(power_data, axis=1)  # shape (n_subjects, n_freqs, n_times)
            # Reshape to (n_subjects, n_freqs * n_times)
            X_power = power_data_subs.reshape(power_data_subs.shape[0], -1)
            # Run permutation t-test across subjects
            T0_pow, p_values_pow, H0_pow = permutation_t_test(X_power, n_permutations=100000, tail=0, n_jobs=None) # two-tailed
            # Reshape p_values back to (n_freqs, n_times) 
            p_values_pow = p_values_pow.reshape(power_data_subs.shape[1], power_data_subs.shape[2])

            # Save the results
            np.save(os.path.join(tfr_group_dir, f"TFR_{group}{task}{task_stage}{block_name}_T_obs.npy"), T0_pow)
            np.save(os.path.join(tfr_group_dir, f"TFR_{group}{task}{task_stage}{block_name}_p_values.npy"), p_values_pow)
            np.save(os.path.join(tfr_group_dir, f"TFR_{group}{task}{task_stage}{block_name}_H0.npy"), H0_pow)

            # PLOT POWER STATS
            # Define significance threshold
            alpha = 0.05
            significant_mask_power = p_values_pow < alpha
            # Make masked array: keep data where significant, mask otherwise
            masked_data_power = np.ma.masked_where(~significant_mask_power, power_data_ave)

            fig, ax = plt.subplots(figsize=(9, 6))
            # Nonsignificant in grayscale
            im1_power = ax.pcolormesh(power.times, power.freqs, power_data_ave,
                                cmap="gray", shading="auto")
            # Significant in color
            im2_power = ax.pcolormesh(power.times, power.freqs, masked_data_power,
                                cmap="summer", shading="auto",
                                vmin=-4, vmax=4)
            ax.set_xlabel("Time (s)")
            ax.set_ylabel("Frequency (Hz)")
            ax.set_title(f"Significant Time-Frequency Power: {group}{task}{task_stage}{block_name}")
            # pick 10 evenly distributed values from freqs
            yticks = np.linspace(0, len(power.freqs) - 1, 10, dtype=int)
            ytick_vals = power.freqs[yticks]
            # Match MNE's frequency scaling
            ax.set_yscale("log")
            ax.set_yticks(ytick_vals)
            ax.set_yticklabels([f"{f:.2f}" for f in ytick_vals])
            ax.get_yaxis().set_major_formatter(plt.ScalarFormatter())
            ax.tick_params(axis='y', which='minor', left=False)  # hide minor log ticks
            fig.colorbar(im2_power, ax=ax, label="Power")
            plt.show()

            fig_title_power = f"{group}{task}{task_stage}{block_name}_power_stats.png"
            fig.savefig(os.path.join(tf_stats_figs_save_path, fig_title_power), dpi=300)
            plt.close('all')


            ### ITC TFR  STATS ###
            itc_data = np.array(itc_data) # shape (n_subjects, n_channels, n_freqs, n_times)
            # Shape before averaging = (n_subjects, n_channels, n_freqs, n_times)
            itc_data_ave = np.mean(itc_data, axis=(0,1)) # average across subs and channels
            # Average across channels but keep subjects
            itc_data_subs = np.mean(itc_data, axis=1)  # shape (n_subjects, n_freqs, n_times)
            # Reshape to (n_subjects, n_freqs * n_times)
            X_itc = itc_data_subs.reshape(itc_data_subs.shape[0], -1)
            # Run permutation t-test across subjects
            T0_itc, p_values_itc, H0_itc = permutation_t_test(X_itc, n_permutations=100000, tail=1, n_jobs=None) # one-tailed (greater than 0)
            # Reshape p_values back to (n_freqs, n_times)
            p_values_itc = p_values_itc.reshape(itc_data_subs.shape[1], itc_data_subs.shape[2])

            # Save the results
            np.save(os.path.join(tfr_group_dir, f"ITC_{group}{task}{task_stage}{block_name}_T_obs.npy"), T0_itc)
            np.save(os.path.join(tfr_group_dir, f"ITC_{group}{task}{task_stage}{block_name}_p_values.npy"), p_values_itc)
            np.save(os.path.join(tfr_group_dir, f"ITC_{group}{task}{task_stage}{block_name}_H0.npy"), H0_itc)

            # PLOT ITC STATS
            # Define significance threshold
            significant_mask_itc = p_values_itc < alpha
            # Make masked array: keep data where significant, mask otherwise
            masked_data_itc = np.ma.masked_where(~significant_mask_itc, itc_data_ave)

            fig, ax = plt.subplots(figsize=(9, 6))
            # Nonsignificant in grayscale
            im1_itc = ax.pcolormesh(itc.times, itc.freqs, itc_data_ave,
                                cmap="gray", shading="auto")
            # Significant in color
            im2_itc = ax.pcolormesh(itc.times, itc.freqs, masked_data_itc,
                                cmap="autumn", shading="auto",
                                vmin=0., vmax=0.15)
            ax.set_xlabel("Time (s)")
            ax.set_ylabel("Frequency (Hz)")
            ax.set_title(f"Significant Time-Frequency ITC: {group}{task}{task_stage}{block_name}")
            # pick 10 evenly distributed values from freqs
            yticks = np.linspace(0, len(itc.freqs) - 1, 10, dtype=int)
            ytick_vals = itc.freqs[yticks]
            # Match MNE's frequency scaling
            ax.set_yscale("log")
            ax.set_yticks(ytick_vals)
            ax.set_yticklabels([f"{f:.2f}" for f in ytick_vals])
            ax.get_yaxis().set_major_formatter(plt.ScalarFormatter())
            ax.tick_params(axis='y', which='minor', left=False)  # hide minor log ticks
            fig.colorbar(im2_itc, ax=ax, label="ITC")
            plt.show()

            fig_title_itc = f"{group}{task}{task_stage}{block_name}_itc_stats.png"
            fig.savefig(os.path.join(tf_stats_figs_save_path, fig_title_itc), dpi=300)
            plt.close('all')


Reading D:\BonoKat\research project\# study 1\eeg_data\set\O\s1_pac_sub12\preproc\analysis\s1_pac_sub12_BL_plan_power-tfr.h5 ...
Reading D:\BonoKat\research project\# study 1\eeg_data\set\O\s1_pac_sub12\preproc\analysis\s1_pac_sub12_BL_plan_itc-tfr.h5 ...
Reading D:\BonoKat\research project\# study 1\eeg_data\set\O\s1_pac_sub14\preproc\analysis\s1_pac_sub14_BL_plan_power-tfr.h5 ...
Reading D:\BonoKat\research project\# study 1\eeg_data\set\O\s1_pac_sub14\preproc\analysis\s1_pac_sub14_BL_plan_itc-tfr.h5 ...
Reading D:\BonoKat\research project\# study 1\eeg_data\set\O\s1_pac_sub15\preproc\analysis\s1_pac_sub15_BL_plan_power-tfr.h5 ...
Reading D:\BonoKat\research project\# study 1\eeg_data\set\O\s1_pac_sub15\preproc\analysis\s1_pac_sub15_BL_plan_itc-tfr.h5 ...
Reading D:\BonoKat\research project\# study 1\eeg_data\set\O\s1_pac_sub17\preproc\analysis\s1_pac_sub17_BL_plan_power-tfr.h5 ...
Reading D:\BonoKat\research project\# study 1\eeg_data\set\O\s1_pac_sub17\preproc\analysis\s1_pac_sub17

__________________________

Dirty field

Stats drafts

In [ ]:
# Shape before averaging = (n_subjects, n_channels, n_freqs, n_times)
power_data_ave = np.mean(power_data, axis=(0,1)) # average across subs and channels

# Average across channels but keep subjects
power_data_subs = np.mean(power_data, axis=1)  # shape (n_subjects, n_freqs, n_times)

# Reshape to (n_subjects, n_freqs * n_times)
X_power = power_data_subs.reshape(power_data_subs.shape[0], -1)

# Run permutation t-test across subjects
T0_pow, p_values_pow, H0_pow = permutation_t_test(X_power, n_permutations=100000, n_jobs=None)

# Reshape p_values back to (n_freqs, n_times)
p_values_pow = p_values_pow.reshape(power_data_subs.shape[1], power_data_subs.shape[2])


Permuting 99999 times...


In [ ]:
# Shape before averaging = (n_subjects, n_channels, n_freqs, n_times)
itc_data_ave = np.mean(itc_data, axis=(0,1)) # average across subs and channels

# Average across channels but keep subjects
itc_data_subs = np.mean(itc_data, axis=1)  # shape (n_subjects, n_freqs, n_times)

# Reshape to (n_subjects, n_freqs * n_times)
X_itc = itc_data_subs.reshape(itc_data_subs.shape[0], -1)

# Run permutation t-test across subjects
T0_itc, p_values_itc, H0_itc = permutation_t_test(X_itc, n_permutations=100000, n_jobs=None)

# Reshape p_values back to (n_freqs, n_times)
p_values_itc = p_values_itc.reshape(itc_data_subs.shape[1], itc_data_subs.shape[2])

Permuting 99999 times...


In [ ]:
# Define significance threshold
alpha = 0.05
significant_mask_power = p_values_pow < alpha

# Make masked array: keep data where significant, mask otherwise
masked_data_power = np.ma.masked_where(~significant_mask_power, power_data_ave)

fig, ax = plt.subplots(figsize=(9, 6))

# Nonsignificant in grayscale
im1_power = ax.pcolormesh(power.times, power.freqs, power_data_ave,
                    cmap="gray", shading="auto")

# Significant in color
im2_power = ax.pcolormesh(power.times, power.freqs, masked_data_power,
                    cmap="summer", shading="auto",
                    vmin=-4, vmax=4)

ax.set_xlabel("Time (s)")
ax.set_ylabel("Frequency (Hz)")
ax.set_title(f"Significant Time-Frequency Power: {group}{task}{task_stage}{block_name}")

# pick 10 evenly distributed values from freqs
yticks = np.linspace(0, len(freqs) - 1, 10, dtype=int)
ytick_vals = power.freqs[yticks]

# # Match MNE's frequency scaling
ax.set_yscale("log")
ax.set_yticks(ytick_vals)
ax.set_yticklabels([f"{f:.2f}" for f in ytick_vals])
ax.get_yaxis().set_major_formatter(plt.ScalarFormatter())
ax.tick_params(axis='y', which='minor', left=False)  # hide minor log ticks


fig.colorbar(im2_power, ax=ax, label="Power")
plt.show()

fig_title_power = f"{group}{task}{task_stage}{block_name}_significant_power.png"
fig.savefig(fig_title_power, dpi=300)
plt.close('all')


In [ ]:
# Define significance threshold
alpha = 0.05
significant_mask_itc = p_values_itc < alpha

# Make masked array: keep data where significant, mask otherwise
masked_data_itc = np.ma.masked_where(~significant_mask_itc, itc_data_ave)

fig, ax = plt.subplots(figsize=(9, 6))

# Nonsignificant in grayscale
im1_itc = ax.pcolormesh(itc.times, itc.freqs, itc_data_ave,
                    cmap="gray", shading="auto")

# Significant in color
im2_itc = ax.pcolormesh(itc.times, itc.freqs, masked_data_itc,
                    cmap="autumn", shading="auto",
                    vmin=0., vmax=0.2)

ax.set_xlabel("Time (s)")
ax.set_ylabel("Frequency (Hz)")
ax.set_title(f"Significant Time-Frequency ITC: {group}{task}{task_stage}{block_name}")

# pick 10 evenly distributed values from freqs
yticks = np.linspace(0, len(freqs) - 1, 10, dtype=int)
ytick_vals = itc.freqs[yticks]

# # Match MNE's frequency scaling
ax.set_yscale("log")
ax.set_yticks(ytick_vals)
ax.set_yticklabels([f"{f:.2f}" for f in ytick_vals])
ax.get_yaxis().set_major_formatter(plt.ScalarFormatter())
ax.tick_params(axis='y', which='minor', left=False)  # hide minor log ticks


fig.colorbar(im2_itc, ax=ax, label="ITC")
plt.show()

fig_title_power = f"{group}{task}{task_stage}{block_name}_significant_itc.png"
fig.savefig(fig_title_power, dpi=300)
# plt.close('all')


______________________

**SINGLE SUB**

In [ ]:
group = 'Y'
sub_name = 'ALL_subs'
task = '_MAIN' # ['_BL', '_MAIN']
task_stage = '_go' # '_plan' or '_go'
block_name = '_adaptation' # ['_baseline', '_adaptation']

group_save_path = os.path.join('D:\\BonoKat\\research project\\# study 1\\eeg_data\\set', f'{group} group')

figs_dir = os.path.join('D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\figures', group, 'epochs', task, 'TFR', block_name)
check_paths(figs_dir)

epochs_all_subs = mne.read_epochs(os.path.join(group_save_path, f"{group}_{sub_name}{task}_epochs{task_stage}{block_name}_ALL-epo.fif"), preload=True)
epochs_all_subs

Reading D:\BonoKat\research project\# study 1\eeg_data\set\Y group\Y_ALL_subs_MAIN_epochs_go_adaptation_ALL-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 60) active
    Found the data of interest:
        t =    -500.00 ...     700.00 ms
        0 CTF compensation matrices available
Adding metadata with 16 columns
4422 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated


<EpochsFIF | 4422 events (all good), -0.5 – 0.7 s (baseline -0.5 – 0.7 s), ~1.21 GB, data loaded, with metadata,
 'go_on': 4422>

In [ ]:
freqs = np.logspace(*np.log10([4, 80]), num=40)
n_cycles = freqs / 2.0  # different number of cycle per frequency
baseline_tf = (-0.5, -0.1)

power, itc = epochs_all_subs.compute_tfr(
    method="morlet",
    freqs=freqs,
    n_cycles=n_cycles,
    average=True,
    return_itc=True,
    decim=3,
    n_jobs=-1
)
power.apply_baseline(baseline=baseline_tf, mode="zscore")

# Save time-freq analysis output
power.save(os.path.join(group_save_path, f"{group}_{sub_name}{task}{task_stage}{block_name}_power-tfr.h5"), overwrite=True)
itc.save(os.path.join(group_save_path, f"{group}_{sub_name}{task}{task_stage}{block_name}_itc-tfr.h5"), overwrite=True)

# plot ITC and Power
itc_fig = itc.plot(combine='mean', title=f'Group: {group}: Target-reaching task: {task_stage[1:]}{block_name}. ITC', cmap='magma')[0]
power_all_fig = power.plot(mode="zscore", title=f'Group: {group}: Target-reaching task: {task_stage[1:]}{block_name}. Averaged Power', combine='mean', cmap='magma')[0]

# save ITC and Power plots
itc_fig.savefig(os.path.join(figs_dir, f"{group}_{sub_name}{task}{task_stage}{block_name}_itc.png"), dpi=300)
power_all_fig.savefig(os.path.join(figs_dir, f"{group}_{sub_name}{task}{task_stage}{block_name}_power_all.png"), dpi=300)

# plt.close('all')


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  54 out of  60 | elapsed:   23.5s remaining:    2.5s


Applying baseline correction (mode: zscore)


[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   24.2s finished


No baseline correction applied
No baseline correction applied


In [ ]:
figs_dir

'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\figures\\Y\\epochs\\_MAIN\\TFR\\_baseline'